In [1]:
import pandas as pd
import numpy as np
from itertools import chain

import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K
import warnings

In [ ]:
# wikidata = pd.read_csv('/Users/rohitmacherla/Downloads/wikihowAll.csv')
# wikidata.shape
# wikidata.head()
# import random
# samples = random.sample(list(wikidata.index), 25000)
# sampled_data = wikidata.loc[samples]
# sampled_data.shape
# test_samples = random.sample(list(sampled_data.index), 5000)
# train_samples = list(set(sampled_data.index) - set(test_samples))
# test_data = sampled_data.loc[test_samples]
# train_data = sampled_data.loc[train_samples]
# train_data = train_data.dropna()
# train_data.to_csv('wikihow_train.csv', index=False)
# test_data.to_csv('wikihow_test.csv', index=False)
# test_data.dropna()

In [7]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/wikihow_train.csv')
train_data = train_data.dropna()
train_data

,Unnamed: 0,headline,title,text
0,196609,"\nTry an acidic cleaner.,\nRemember that stron...",How to Improve a Toilet's Flushing Power3,Very old toilets may accumulate hard water de...
1,4,"\nKeep your reference materials, sketches, art...",How to Be an Organized Artist2,"As you start planning for a project or work, ..."
2,196614,"\nLook at the bigger picture.,\nExamine your f...",How to Adjust Toilet Tank Handles,The flush handle is just one part of the syst...
3,196620,"\nApply lemon Kool-aid.,\nUse a pumice stone.,...",How to Clean a Stained Toilet Bowl2,Hard water stains are both common and unsight...
4,196622,"\nSelect which product you'd like to use.,\nCr...",How to Clean a Stained Toilet Bowl4,Mildew in your home is linked to respiratory ...
...,...,...,...,...
19995,131051,"\nPre-cook your veggies.,\nGo outside your veg...",How to Make Salads More Flavorful1,Grill your vegetables before adding them to y...
19996,65518,"\nKeep a close eye on your dog.,\nUse tightly ...",How to Care for a Skunk Sprayed Dog3,"If you have a very large, open backyard, cons..."
19997,196591,"\nShut off the toilet’s water supply.,\nFill t...",How to Increase Water Pressure in a Toilet3,Locate the shutoff valve and turn it clockwis...
19998,65521,"\nPut on a latex glove.,\nInflate a balloon.,\...",How to Remove Pet Hair3,Rub the surface with it on. The pet hair shou...


In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/wikihow_test.csv')
test_data = test_data.dropna()
test_data

,Unnamed: 0,headline,title,text
0,14285,"\nChange the way you chew.,\nDo not chew very ...",How to Chew Gum in Class,Chew slowly and discreetly. Make sure your mo...
1,138630,\nLaunch the Skype app.Sign in with your usern...,How to Invite Someone on Skype2,", The icon looks like an address book and appe..."
2,101875,"\nEnable the Developer tab.,\nClick the More C...",How to Embed YouTube Flash Videos in Your Powe...,The Developer tab will allow you to insert ob...
3,60326,"\nEvaluate your social media use.,\nAvoid comp...",How to Know if You're Rebounding or if You Act...,"In our modern age, ending a relationship may ..."
4,95644,\nOpen your web browser and visit videolan.org...,How to Play VOB Files1,"VLC Player is a free, open-source video playe..."
...,...,...,...,...
4995,115599,"\nKnow your fish.,\nPut some serious thought i...",How to Keep Japanese Weather Loaches,Weather loaches (Dojo Loaches) aren't an ordi...
4996,192675,\nStart outline sketch for Liam Payne’s head.\...,How to Draw One Direction3,",,,,,"
4997,180807,"\nBe old enough.,\nMeet the citizenship requir...",How to Become a Canadian Police Officer,You must be at least 18 years old in most are...
4998,783,"\nStay calm.,\nTake action early on.,\nKnow th...",How to Get Someone out of Your Head3,"Bullies love a reaction, and they particularl..."


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))
def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:
            long_words.append(i)
    return (" ".join(long_words)).strip()

cleaned_text = []
for t in train_data['text']:
    cleaned_text.append(text_cleaner(t))

In [ ]:
def summary_cleaner(text):
    newString = re.sub('"','', text)
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower()
    tokens=newString.split()
    newString=''
    for i in tokens:
        if len(i)>1:
            newString=newString+i+' '
    return newString

cleaned_summary = []
for t in train_data['headline']:
    cleaned_summary.append(summary_cleaner(t))

train_data['cleaned_text']=cleaned_text
train_data['cleaned_summary']=cleaned_summary
train_data['cleaned_summary'].replace('', np.nan, inplace=True)
train_data.dropna(axis=0,inplace=True)

In [ ]:
X_train = train_data['cleaned_text']
y_train = train_data['cleaned_summary']

In [ ]:
cleaned_text_test = []
for t in test_data['text']:
    cleaned_text_test.append(text_cleaner(t))

In [ ]:
cleaned_summary_test = []
for t in test_data['headline']:
    cleaned_summary_test.append(summary_cleaner(t))

In [ ]:
test_data['cleaned_text']=cleaned_text_test
test_data['cleaned_summary']=cleaned_summary_test
test_data['cleaned_summary'].replace('', np.nan, inplace=True)
test_data.dropna(axis=0,inplace=True)

In [ ]:
X_test = test_data['cleaned_text']
y_test = test_data['cleaned_summary']

In [ ]:
tokenizer_doc = Tokenizer()
tokenizer_doc.fit_on_texts(X_train)
sequences_doc = tokenizer_doc.texts_to_sequences(X_train)

tokenizer_sum = Tokenizer()
tokenizer_sum.fit_on_texts(y_train)
sequences_sum = tokenizer_sum.texts_to_sequences(y_train)

max_len_text = max(map(len, sequences_doc))
max_len_summary = max(map(len, sequences_sum))

x_voc_size = len(tokenizer_doc.word_index) + 1
y_voc_size = len(tokenizer_sum.word_index) + 1

X_train_padded = pad_sequences(sequences_doc, maxlen=max_len_text, padding='post')
y_train_padded = pad_sequences(sequences_sum, maxlen=max_len_summary, padding='post')

In [ ]:
embedding_dim = 100
model = Sequential()
model.add(Embedding(x_voc_size, embedding_dim, input_length=max_len_text))
model.add(LSTM(100))
model.add(RepeatVector(max_len_summary))
model.add(LSTM(100, return_sequences=True))
model.add(TimeDistributed(Dense(y_voc_size, activation='softmax')))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 8

model.fit(X_train_padded, y_train_padded, epochs=5, batch_size=batch_size, validation_split=0.2, verbose=True)

Epoch 1/5
1990/1990 [==============================] - 782s 390ms/step - loss: 0.3827 - accuracy: 0.9741 - val_loss: 0.2205 - val_accuracy: 0.9751
Epoch 2/5
1990/1990 [==============================] - 669s 336ms/step - loss: 0.2217 - accuracy: 0.9748 - val_loss: 0.2195 - val_accuracy: 0.9754
Epoch 3/5
1990/1990 [==============================] - 652s 328ms/step - loss: 0.2208 - accuracy: 0.9749 - val_loss: 0.2194 - val_accuracy: 0.9754
Epoch 4/5
1990/1990 [==============================] - 647s 325ms/step - loss: 0.2202 - accuracy: 0.9750 - val_loss: 0.2190 - val_accuracy: 0.9754
Epoch 5/5
1990/1990 [==============================] - 643s 323ms/step - loss: 0.2196 - accuracy: 0.9750 - val_loss: 0.2195 - val_accuracy: 0.9754


In [ ]:
model.save("/content/drive/MyDrive/wikihow-model2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
model = tf.keras.models.load_model("/content/drive/MyDrive/Models/wikihow-lstm.h5")

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2667, 100)         5923500   
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 repeat_vector (RepeatVecto  (None, 2261, 100)         0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 2261, 100)         80400     
                                                                 
 time_distributed (TimeDist  (None, 2261, 29743)       3004043   
 ributed)                                                        
                                                                 
Total params: 9088343 (34.67 MB)
Trainable params: 90883

In [12]:
!pip install pydot
!pip install graphviz

In [ ]:
tokenizer_doc = Tokenizer()
tokenizer_doc.fit_on_texts(X_test)
sequences_doc = tokenizer_doc.texts_to_sequences(X_test)

tokenizer_sum = Tokenizer()
tokenizer_sum.fit_on_texts(y_test)
sequences_sum = tokenizer_sum.texts_to_sequences(y_test)

X_test_padded = pad_sequences(sequences_doc, maxlen=max_len_text, padding='post')
y_test_padded = pad_sequences(sequences_sum, maxlen=max_len_summary, padding='post')

In [ ]:
batch_size = 8
num_batches = len(X_test_padded) // batch_size

y_pred_texts = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size

    # Extract a batch of input sequences
    batch_padded_doc = X_test_padded[start_idx:end_idx]

    # Generate predictions for the batch
    batch_predicted_summary_sequences = model.predict(batch_padded_doc)

    # Get word indices
    batch_word_indices = np.argmax(batch_predicted_summary_sequences, axis=-1)

    # Convert indices to words
    batch_y_pred_texts = [tokenizer_sum.sequences_to_texts([seq])[0] for seq in batch_word_indices]

    # Extend the result list
    y_pred_texts.extend(batch_y_pred_texts)

# If there's a remainder (i.e., len(padded_doc) is not a multiple of batch_size), process the remaining samples
if len(X_test_padded) % batch_size != 0:
    remainder_start_idx = num_batches * batch_size
    remainder_padded_doc = X_test_padded[remainder_start_idx:]

    remainder_predicted_summary_sequences = model.predict(remainder_padded_doc)
    remainder_word_indices = np.argmax(remainder_predicted_summary_sequences, axis=-1)
    remainder_y_pred_texts = [tokenizer_sum.sequences_to_texts([seq])[0] for seq in remainder_word_indices]

    y_pred_texts.extend(remainder_y_pred_texts)

1/1 [==============================] - 0s 71ms/step


In [ ]:
test_data['preds'] = y_pred_texts

In [ ]:
test_data.to_csv('/content/drive/MyDrive/wikihow_preds.csv')

In [ ]:
# Set the locale to UTF-8
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=ef5b6c0b7cc074b1ca10473535885c98c8f8ad558a830f175ddfbec0ab9ad939
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# Define a filtering criterion
def is_valid_summary(summary):
    return summary.strip() != '' and len(set(summary)) > 1  # Adjust as needed

# Apply the filter to create a subset of valid summaries
valid_summaries_df = test_data[test_data['preds'].apply(is_valid_summary)]

# Now you can use valid_summaries_df for metrics calculation

In [ ]:

from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

#sampled_df = valid_summaries_df_df.dropna(subset=['generated_summary'])

# Calculate BLEU score
references = valid_summaries_df['headline']
hypotheses = valid_summaries_df['preds']

bleu_score = corpus_bleu([[ref.split()] for ref in references], [hyp.split() for hyp in hypotheses], weights=(0.5, 0.5))
print(f"BLEU Score: {bleu_score * 100:.2f}")

BLEU Score: 0.03


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [ ]:
from rouge_score import rouge_scorer

# Assuming 'reference_summary' and 'hypothesis_summary' are Pandas Series
references = valid_summaries_df['headline'].astype(str).tolist()
hypotheses = valid_summaries_df['preds'].astype(str).tolist()

# Create a RougeScorer instance
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=True)

# Calculate ROUGE scores for each pair of reference and hypothesis
rouge_scores = []
for ref, hyp in zip(references, hypotheses):
    scores = scorer.score(ref, hyp)
    rouge_scores.append(scores)

# Print the scores for the first pair (adjust as needed)
print("ROUGE-1 F1 Score:", rouge_scores[0]['rouge1'].fmeasure)
print("ROUGE-2 F1 Score:", rouge_scores[0]['rouge2'].fmeasure)
print("ROUGE-L F1 Score:", rouge_scores[0]['rougeLsum'].fmeasure)

ROUGE-1 F1 Score: 0.16091954022988506
ROUGE-2 F1 Score: 0.0
ROUGE-L F1 Score: 0.16091954022988506
